In [ ]:
#need for kaggle online ipynb
#! pip install imutils

In [ ]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
import os
import random
import cv2
import imutils
import random
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelBinarizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras import optimizers
from sklearn.preprocessing import LabelBinarizer
from keras import backend as K
from keras.layers import Dense, Activation, Flatten, Dense,MaxPooling2D, Dropout
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import tensorflow as tf

In [ ]:
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

# #make sure to use GPU
# tf.debugging.set_log_device_placement(True)

In [ ]:
# gpus = tf.config.list_physical_devices('GPU')
# gpus

In [ ]:
# for google colab - after you load your kaggle api token you can give it permissions on colab (if using colab)
# !chmod 600 /content/kaggle.json

In [ ]:
# google colab 
# ! kaggle datasets download -d vaibhao/handwritten-characters
# ! kaggle datasets download -d landlord/handwriting-recognition

In [ ]:
# import zipfile #unzipping the dataset (google colab)
# zip_ref = zipfile.ZipFile('handwritten-characters.zip', 'r')
# zip_ref.extractall('/content/datasets/handwritten-characters')
# zip_ref.close()

In [ ]:
# import zipfile #unzipping the dataset (google colab)
# zip_ref = zipfile.ZipFile('handwriting-recognition.zip', 'r')
# zip_ref.extractall('/content/datasets/handwriting-recognition')
# zip_ref.close()

Team used next datasets: <br>
In first iteration (those two are not used in final model, but can be used for handwritten text recognition):
- [Handwritten Characters] https://www.kaggle.com/datasets/vaibhao/handwritten-characters/
- [Handwriting Recognition] https://www.kaggle.com/datasets/landlord/handwriting-recognition

In second iteration (used in final model):
- [Printed Digits] https://www.kaggle.com/datasets/dhruvmomoman/printed-digits
- [Camera-Taken Images of Printed English Alphabet] https://www.kaggle.com/datasets/naderabdalghani/camerataken-images-of-printed-english-alphabet

In [ ]:
# for kaggle (online model):
# dir = "/kaggle/input/printed-digits/"
# dir2 = "/kaggle/input/camerataken-images-of-printed-english-alphabet/"

# for local model:
dir = "datasets/printed-digits/"
dir2 = "datasets/camerataken-images-of-printed-english-alphabet/"

dataset_data = []
img_size = 28

non_chars = ["ImageMap.txt"]
limit_images = 400 #limiting the number of images per label, set to more than 1000 to get all images

def get_data(dir, max_images, not_list):
    for i in os.listdir(dir):
        if i in not_list:
            continue
        count = 0
        sub_directory = os.path.join(dir,i)
        for j in os.listdir(sub_directory):
            count+=1
            if count > max_images: #limiting the number of images per label
                break
            img = cv2.imread(os.path.join(sub_directory,j),0)
            img = cv2.resize(img,(img_size,img_size))
            dataset_data.append([img,i])

get_data(dir, limit_images, non_chars)
get_data(dir2, limit_images, non_chars)

len(dataset_data) # must be 5650 + 47,981 = 53,631 images if not limited. 31k if limited to 500 images per label

In [ ]:
#splitting the data into train, val and test
from sklearn.model_selection import train_test_split

X = []
y = []

for features, label in dataset_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, img_size, img_size, 1)
y = np.array(y)

#remove _ from labels
y = [i.replace("_", "") for i in y]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101, stratify=y)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=101)

In [ ]:
#normalizing the data
X_train = X_train/255
X_test = X_test/255
X_val = X_val/255

In [ ]:
#one hot encoding the labels
LB = LabelBinarizer()
y_train = LB.fit_transform(y_train)
y_test = LB.fit_transform(y_test)
y_val = LB.fit_transform(y_val)

In [ ]:
LB.classes_

In [ ]:
# features
print(X_train.shape,X_val.shape, X_test.shape)

In [ ]:
# labels
print(y_train.shape,y_val.shape, y_test.shape)

**Experiments with models and layers**
- MaxPooling2D to use over AveragePooling2D as it is more robust to noise.
- Dropout to reduce overfitting.
- BatchNormalization to normalize the activations of the previous layers.

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), strides=(1, 1), activation='relu', padding='same', kernel_initializer='he_normal', input_shape=(img_size, img_size,1)),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.SpatialDropout2D(0.4),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(64, (3, 3), strides=(1, 1), activation='relu', padding='same', kernel_initializer='he_normal'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.SpatialDropout2D(0.4),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Conv2D(128, (3, 3), strides=(1, 1), activation='relu', padding='same', kernel_initializer='he_normal'),
    tf.keras.layers.SpatialDropout2D(0.4),
    tf.keras.layers.BatchNormalization(),
#    tf.keras.layers.GlobalMaxPooling2D(),
#     tf.keras.layers.Reshape((128, 1)), #these 4 layers are for the RNN and increase time taken to train by a lit 2min -> 25min per epoch on CPU
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128, return_sequences=True)),
#     tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(128)),
#    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(units=128, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.Dense(units=128, activation='relu', kernel_initializer='he_normal'),
    tf.keras.layers.Dropout(0.4),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(units=62, activation='softmax')
])

model.compile(loss = 'categorical_crossentropy', optimizer = "adam", metrics = ['accuracy'])
model.summary()

In [ ]:
#currently in testing purposes we do not need to run a lot of epochs to avoid overfitting
es = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 2, restore_best_weights=True)
history = model.fit(X_train,y_train, epochs=5, batch_size=32, validation_data = (X_val, y_val),  verbose=1, callbacks=[es])

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Training Accuracy vs Validation Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Training Loss vs Validation Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
#testing the model
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=1)
print('Test loss:', test_loss)
print('Test accuracy:', test_acc)

#save model
model.save("textReadModel.keras")

In [ ]:
#Viewing some predictions
for i in range(10):
    index = random.randint(0, len(X_test))
    plt.imshow(X_test[index].reshape(img_size,img_size), cmap='gray')
    pred = model.predict(X_test[index].reshape(1,img_size,img_size,1))
    print("Predicted: " + LB.classes_[np.argmax(pred)])
    print("Actual: " + LB.classes_[np.argmax(y_test[index])])
    plt.show()